Условие: есть набор данных о продажах продуктов с информацией о дате продаж, категории продукта, количестве и выручке от продаж.

Используя Apache Spark, загрузите предоставленный набор данных в DataFrame (пример данных ниже).

("2023-11-20", "Electronics", 100, 12000),
("2023-11-21", "Electronics", 110, 13000),
("2023-11-22", "Electronics", 105, 12500),
("2023-11-20", "Clothing", 300, 15000),
("2023-11-21", "Clothing", 280, 14000),
("2023-11-22", "Clothing", 320, 16000),
("2023-11-20", "Books", 150, 9000),
("2023-11-21", "Books", 200, 12000),
("2023-11-22", "Books", 180, 10000)

Столбцы: "date", "category", "quantity", "revenue".

С использованием оконных функций, рассчитайте среднее выручки от продаж для каждой категории продукта.
Примените операцию pivot для того, чтобы преобразовать полученные данные таким образом, чтобы в качестве строк были категории продуктов, в качестве столбцов были дни, а значениями были средние значения выручки от продаж за соответствующий день

In [ ]:
#Устанавливаем pySpark
!pip install pyspark

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import Window
from pyspark.sql.functions import *

In [3]:
# Создаем SparkSession
spark = SparkSession.builder.appName("Sales Analysis").getOrCreate()

In [8]:
# Создаем DataFrame
data = [("2023-11-20", "Electronics", 100, 12000),
        ("2023-11-21", "Electronics", 110, 13000),
        ("2023-11-22", "Electronics", 105, 12500),
        ("2023-11-20", "Clothing", 300, 15000),
        ("2023-11-21", "Clothing", 280, 14000),
        ("2023-11-22", "Clothing", 320, 16000),
        ("2023-11-20", "Books", 150, 9000),
        ("2023-11-21", "Books", 200, 12000),
        ("2023-11-22", "Books", 180, 10000)]

columns = ["date", "category", "quantity", "revenue"]
df = spark.createDataFrame(data, columns)

df.show()

+----------+-----------+--------+-------+
|      date|   category|quantity|revenue|
+----------+-----------+--------+-------+
|2023-11-20|Electronics|     100|  12000|
|2023-11-21|Electronics|     110|  13000|
|2023-11-22|Electronics|     105|  12500|
|2023-11-20|   Clothing|     300|  15000|
|2023-11-21|   Clothing|     280|  14000|
|2023-11-22|   Clothing|     320|  16000|
|2023-11-20|      Books|     150|   9000|
|2023-11-21|      Books|     200|  12000|
|2023-11-22|      Books|     180|  10000|
+----------+-----------+--------+-------+



In [10]:
# Создаем окно для среднего значения
windowSpec = Window.partitionBy("category", "date")

# Расчитываем среднее значение
avg_df = df.withColumn("average_revenue", mean("revenue").over(windowSpec))

# Применяем pivot
pivot_df = avg_df.groupBy("category").pivot("date").agg(first("average_revenue"))

# Показываем результат
pivot_df.show()


+-----------+----------+----------+----------+
|   category|2023-11-20|2023-11-21|2023-11-22|
+-----------+----------+----------+----------+
|Electronics|   12000.0|   13000.0|   12500.0|
|   Clothing|   15000.0|   14000.0|   16000.0|
|      Books|    9000.0|   12000.0|   10000.0|
+-----------+----------+----------+----------+

